In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv("oil_production.csv")
df1 = pd.read_csv("forecast.csv")

x = df.describe()

from sklearn.preprocessing import MinMaxScaler, Normalizer, MaxAbsScaler, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, LassoCV, ElasticNet, SGDRegressor, Lasso

from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor

import plotly.graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected = True)
import plotly.io as pio

from scipy.stats import pearsonr as r
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

test_fraction = 0.15
val_fraction = 0.15
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = test_fraction, shuffle = False, random_state = 42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size 
                                                  = val_fraction/(1-test_fraction), shuffle = False, random_state = 1000)

scaler_x = MinMaxScaler()
x_train = scaler_x.fit_transform(x_train.as_matrix())
x_val = scaler_x.transform(x_val.as_matrix())
x_test = scaler_x.transform(x_test.as_matrix())

scaler_y = MinMaxScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1,1))
y_val = scaler_y.transform(y_val.reshape(-1,1))
y_test = scaler_y.transform(y_test.reshape(-1,1))

MLP = MLPRegressor(hidden_layer_sizes=(50,),  activation='relu', solver='adam', alpha=0.0001, batch_size=200,
    learning_rate='adaptive', learning_rate_init=0.001, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=9, tol=0.0001, verbose=False, warm_start=False, momentum=0.5, nesterovs_momentum=True,
    early_stopping=True, validation_fraction=0.15, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
MLP.fit(x_train, y_train)
MLP.score(x_val, y_val)
MLP.score(x_test, y_test)
MLP.predict(x_test)
prediction = MLP.predict(x_test)
prediction.min()

y_pred_mlp = scaler_y.inverse_transform(MLP.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_mlp.reshape(-1,),
                   name = "Actual",
                   mode = 'markers',
                   line=dict(width=10))
data = [trace1]
layout = go.Layout(
    xaxis=dict(
        title='Actual oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted oil rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

print(mse(y_val, MLP.predict(x_val)))
print(mse(y_test, MLP.predict(x_test)))
r(y_test, MLP.predict(x_test).reshape(-1,1))[0]



def create_timeblock(X, Y, look_back=1):
    dataX, dataY = [], []
    for i in range(len(X) - look_back):
        a = X[i:(i + look_back), :]
        dataX.append(a)
        dataY.append(Y[i + look_back, :])
    print('# of data: ', len(dataY))
    return np.array(dataX), np.array(dataY)

test_fraction = 0.15
X = df[['ON_STREAM_HRS','AVG_DOWNHOLE_PRESSURE', 'AVG_DOWNHOLE_TEMPERATURE',
        'AVG_DP_TUBING', 'AVG_CHOKE_SIZE_P', 'AVG_WHP_P','DP_CHOKE_SIZE', 'AVG_WHT_P']]
Y = df[['BORE_OIL_VOL', 'BORE_GAS_VOL','BORE_WAT_VOL']]

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = test_fraction, shuffle = False)
val_fraction = 0.15
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train,
                                                  test_size = val_fraction/(1-test_fraction), shuffle = False)


scaler_x = MinMaxScaler()
x_train_norm = scaler_x.fit_transform(x_train.as_matrix())
x_val_norm = scaler_x.transform(x_val.as_matrix())
x_test_norm = scaler_x.transform(X_test.as_matrix())

scaler_y = MinMaxScaler()
y_train_norm = scaler_y.fit_transform(y_train.as_matrix())
y_val_norm = scaler_y.transform(y_val.as_matrix())
y_test_norm = scaler_y.transform(Y_test.as_matrix())

x_train, y_train = create_timeblock(x_train_norm, y_train_norm, look_back=35)
x_val, y_val = create_timeblock(x_val_norm, y_val_norm, look_back=35)
x_test, y_test = create_timeblock(x_test_norm, y_test_norm, look_back=35)


def run_model(x_train, y_train, x_val, y_val, epochs=500, batch_size=400, method='LSTM'):
    classifier = {'GRU' : GRU,
                  'LSTM' : LSTM,
                  'Simple RNN' : SimpleRNN}


    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))

    X = classifier[method](512, return_sequences=True)(inputs)
    X = classifier[method](1024, return_sequences=True)(X)
    X = classifier[method](512, return_sequences=True)(X)
    #X = classifier[method](256, return_sequences=True)(X)
    X = TimeDistributed(Dense(256, activation='relu'))(X)
    X = Dropout(0.4)(X)
    
    out1 = Dense(1, activation='linear')(Flatten()(X))
    out2 = Dense(1, activation='linear')(Flatten()(X))
    out3 = Dense(1, activation='linear')(Flatten()(X))
    
    model = Model(input=inputs, output=[out1, out2, out3])
    opt = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.001)

    #parallel_model = multi_gpu_model(model, gpus=8)

    model.compile(loss='mean_squared_error', optimizer= opt, metrics=['mae'])
    history = model.fit(x_train, [y_train[:, 0], y_train[:, 1], y_train[:, 2]], 
                        epochs=epochs, batch_size=batch_size,
                        validation_data=(x_val, [y_val[:, 0], y_val[:, 1], y_val[:, 2]]), 
                        verbose=1, shuffle=False)
    return model, history

# Training
y_pred_train_pre = model_LSTM.predict(x_train)
y_pred_train_scaled = np.concatenate((y_pred_train_pre[0], y_pred_train_pre[1], y_pred_train_pre[2]), axis=1)
y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled)
y_train_act = scaler_y.inverse_transform(y_train)

# Dev
y_pred_dev_pre = model_LSTM.predict(x_val)
y_pred_dev_scaled = np.concatenate((y_pred_dev_pre[0], y_pred_dev_pre[1], y_pred_dev_pre[2]), axis=1)
y_pred_dev = scaler_y.inverse_transform(y_pred_dev_scaled)
y_dev_act = scaler_y.inverse_transform(y_val)

# Test
y_pred_test_pre = model_LSTM.predict(x_test)
y_pred_test_scaled = np.concatenate((y_pred_test_pre[0], y_pred_test_pre[1], y_pred_test_pre[2]), axis=1)
y_pred_test = scaler_y.inverse_transform(y_pred_test_scaled)
y_test_act = scaler_y.inverse_transform(y_test)

# Train-Dev-Test
y_act = np.concatenate([y_train_act, y_dev_act, y_test_act])
y_pred = np.concatenate([y_pred_train, y_pred_dev, y_pred_test])

def plot_iterative_flow(t, q_actual, q_pred, val_fraction, test_fraction):
    lowrate_type = {0:"Actual oil rate",
                    1: "Actual gas rate",
                    2: "Actual water rate"}
    trace1 = go.Scatter(x = t,
                        y = q_actual,
                        name = lowrate_type[i])
    train_tail = int((1 -val_fraction - test_fraction) * q_pred.shape[0])
    trace2 = go.Scatter(x = t[0:train_tail],
                        y = q_pred[0:train_tail],
                        name = "Training set")

    val_tail = int((1-test_fraction) * q_pred.shape[0])
    trace3 = go.Scatter(x = t[train_tail:val_tail],
                        y = q_pred[train_tail:val_tail],
                        name = "Val set")
    trace4 = go.Scatter(x = t[val_tail:],
                        y = q_pred[val_tail:],
                        name = "Test set")
    data = [trace1, trace2, trace3, trace4]
    layout = {"title": "Flow rate well"}
    fig = go.Figure(data = data, layout = layout)
    py.iplot(fig)
    
t = np.arange(len(df))
for i in range(y_act.shape[1]):
    plot_iterative_flow(t,
                   y_act[:, i], y_pred[:, i], val_fraction=val_fraction, test_fraction=test_fraction)
S = 0
cum = []
for i in y_pred_train[:,0]:
    S = S + i
    cum.append(S)
for i in y_pred_dev[:,0]:
    S+= i
    cum.append(S)
for i in y_pred_test[:,0]:
    S+= i
    cum.append(S)
    
T = 0
re = []
for i in y_train_act[:,0]:
    T += i
    re.append(T)
for i in y_dev_act[:,0]:
    T += i
    re.append(T)
for i in y_test_act[:,0]:
    T += i
    re.append(T)
a = np.asarray(cum)
b = np.asarray(re)

trace1 = go.Scatter(x = np.arange(len(a)),
                    y = a.reshape(-1,),
                   name = "Prediction")
trace2 = go.Scatter(x = np.arange(len(a)),
                    y = b.reshape(-1,),
                   name = "Actual values")
data = [trace1, trace2]
layout = go.Layout(title = "Cummulative oil production of Long short term memory")
fig = go.Figure(data = data, layout = layout)
py.iplot(fig)